In [ ]:
!pip install bitsandbytes datasets accelerate loralib
!pip install git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [ ]:
!pip install scikit-learn

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

#Локальный путь. Используем, если скачали модель локально ранее
#base_model = "models/download/pythia-12b-deduped"

#Скачиваем базовую модель с Hugging Face 
base_model = "EleutherAI/pythia-12b-deduped"

model = AutoModelForCausalLM.from_pretrained(
    base_model,
      load_in_8bit=True, 
    device_map='auto'
)

tokenizer = AutoTokenizer.from_pretrained(base_model)

In [ ]:
#Добавляем PAD токен

#Выводим список текущих токенов
special_tokens = tokenizer.all_special_tokens
print("Special tokens:", special_tokens)

#Добавляем PAD токен
special_tokens = {'pad_token': '<PAD>'}
tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

# Получаем список токенов после апдейта
special_tokens = tokenizer.all_special_tokens

print("Special tokens:", special_tokens)

In [ ]:
#Загружаем датасет для тренировки модели
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_parquet("path_to_dataset/training_dataset.parquet")

train, test = train_test_split(df, test_size=0.01) 

train.to_parquet('train.parquet', index=False)
test.to_parquet('test.parquet', index=False)

In [ ]:
from datasets import load_dataset

dataset = load_dataset('parquet', data_files={'train': 'train.parquet',
                                              'test': 'test.parquet'}).shuffle()

In [ ]:
max_length = 1024

#В данном примере внутри train датасета данные хранятся в колонке QA, 
#если у вас другое имя колонки, необходимо поменять имя здесь examples["QA"]
def tokenize_function(examples):
    return tokenizer(examples["QA"], padding='max_length', truncation=True, max_length=max_length)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["QA"])
tokenized_datasets.set_format("torch")

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd

from tqdm.auto import tqdm

In [ ]:
#задаем основные параметры для тренировки
#могут влиять на потребление ресурсов, скорость обучения

#Подробнее как тюнить параметры можно почитать по ссылкам:
#https://huggingface.co/blog/peft
#https://arxiv.org/pdf/2106.09685.pdf 

MICRO_BATCH_SIZE = 2  
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 3  
LEARNING_RATE = 3e-4  
LORA_R = 128
LORA_ALPHA = 256
LORA_DROPOUT = 0.01

print(GRADIENT_ACCUMULATION_STEPS)

In [ ]:
from peft import prepare_model_for_int8_training
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules= ["query", "key", "value"]
)

model = prepare_model_for_int8_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
from torch.utils.data import DataLoader

full_train_dataset = tokenized_datasets["train"]
train_dataloader = DataLoader(full_train_dataset, shuffle=True, batch_size=MICRO_BATCH_SIZE)

steps_per_epoch = len(tokenized_datasets["train"]) // (MICRO_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS)
print(steps_per_epoch)

In [ ]:
#Запускам тренировку модели
#В зависимости от размера датасета, тренировка может занимать как минуты так и десятки или сотни часов

import transformers

trainer = transformers.Trainer(
    model=model,
    train_dataset= tokenized_datasets["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=20,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=1,
        save_steps=steps_per_epoch,
        output_dir="lora-pythia",
        save_total_limit=5),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train(resume_from_checkpoint=False)

In [ ]:
#Сохраняем полученную модель
#Для сервинга и тестирования используйте следующую тетрадку 02_peft_serve
import os
import datetime

save_dir = "models/trained/peft/"
os.makedirs(save_dir)
print(save_dir)
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print('Done')